# 아산병원_질병백과_상세정보_크롤링

- 아산병원 질병백과 크롤링에서 수집한 diseases_240510.csv의 질병id, 질병명 활용하여 질문설명 파일 생성

# 관련 모듈 설치

In [40]:
pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB 8.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.8/11.6 MB 9.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.6 MB 10.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.6 MB 11.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/11.6 MB 10.6 MB/s eta 0:00:01
   --------- ------------------------------ 2.8/11.6 MB 11.3 MB/s eta 0:00:01
   ----------- ---------------------------- 3.4/11.6 MB 11.4 MB/s eta 0:00:01
   ------------- -------------------------- 3.9/11.6 MB 11.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.5/11.6 MB 11.4 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.6 MB 11.3 MB/s eta 0:00:01
   ------------------ --------------------- 5.4/11.6 MB 11.1 MB/s eta 0:00:01
   ---

# 관련 모듈 임포트

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# 데이터 준비

In [56]:
# CSV 파일 경로
file_path = './../Model/Data/diseases_240510.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)
                
# disease_id 중복제거
unique_df = df.drop_duplicates(subset=['disease_id'])

In [57]:
unique_df.head(2)

,disease_id,disease_name,disease_link,disease_img,symptoms,symptom_ids,related_diseases,related_disease_ids,department,synonyms
0,31610,RS바이러스 감염증(RS virus infection),https://www.amc.seoul.kr/asan/healthinfo/disea...,https://www.amc.seoul.kr/asan/file/imageView.d...,"기침, 가래, 열, 빈호흡, 콧물, 코막힘, 청색증","SS000868,SS000859,SS000684,SS000875,SS000748,S...","폐렴, 급성 기관지염, 상기도 감염","31631,31810,31816",감염내과,"RSV,RS바이러스,RS바이러스 감염,알에스바이러스,호흡기세포융합바이러스,호흡기세포..."
1,31292,각막염(Keratitis),https://www.amc.seoul.kr/asan/healthinfo/disea...,https://www.amc.seoul.kr/asan/file/imageView.d...,"안구혼탁, 눈의 충혈, 눈물흘림, 눈의 통증, 시력 감소, 눈부심","SS000217,SS000186,SS000181,SS000187,SS000204,S...","각막 혼탁과 흉터, 진균성 각막 궤양, 각막 궤양, 헤르페스 바이러스 감염","31294,32237,32224,31538",안과,각막염증


# 크롤링 및 CSV 파일 저장

In [54]:
# CSV 파일에 저장할 헤더
header = ['disease_id', 'disease_name', 'descDl_definition', 'descDl_cause', 'descDl_symptoms', 'descDl_diagnosis', 'descDl_treatment', 'descDl_course']

# 진도확인용 count
count = 0

# CSV 파일 열기
with open('disease_description.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    # 'disease_id'와 'disease_name' 열만 추출하여 반복문에 사용
    for index, row in unique_df.iterrows():
        try: 
            disease_id = row['disease_id']
            disease_name = row['disease_name']
            url = f'https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId={disease_id}'
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # dt 요소에 따라 각각의 정보를 추출하고, 해당 요소가 없는 경우 빈 문자열로 대체
            descDl_definition_elem = soup.select_one('dt:-soup-contains("정의") + dd')
            descDl_definition = descDl_definition_elem.text if descDl_definition_elem else ""
            
            descDl_cause_elem = soup.select_one('dt:-soup-contains("원인") + dd')
            descDl_cause = descDl_cause_elem.text if descDl_cause_elem else ""
            
            descDl_symptoms_elem = soup.select_one('#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dt:-soup-contains("증상") + dd')
            descDl_symptoms = descDl_symptoms_elem.text if descDl_symptoms_elem else ""
            
            descDl_diagnosis_elem = soup.select_one('dt:-soup-contains("진단") + dd')
            descDl_diagnosis = descDl_diagnosis_elem.text if descDl_diagnosis_elem else ""
            
            descDl_treatment_elem = soup.select_one('dt:-soup-contains("치료") + dd')
            descDl_treatment = descDl_treatment_elem.text if descDl_treatment_elem else ""
            
            descDl_course_elem = soup.select_one('dt:-soup-contains("경과") + dd')
            descDl_course = descDl_course_elem.text if descDl_course_elem else ""
            # CSV 파일에 추가
            writer.writerow([disease_id, disease_name, descDl_definition, descDl_cause, descDl_symptoms, descDl_diagnosis, descDl_treatment, descDl_course])
            count += 1
            print(count, disease_id, " 끝")
        except:
            print(id, e)

31610  끝
31292  끝
31708  끝
31618  끝
32450  끝
31604  끝
32850  끝
32223  끝
32191  끝
31719  끝
31727  끝
31672  끝
31741  끝
31740  끝
31546  끝
31609  끝
31632  끝
32151  끝
32160  끝
31661  끝
31725  끝
31827  끝
33905  끝
33444  끝
33451  끝
32751  끝
33710  끝
32162  끝
32296  끝
33912  끝
32838  끝
31541  끝
31888  끝
31617  끝
32161  끝
32297  끝
32778  끝
31852  끝
31597  끝
33614  끝
31664  끝
33891  끝
31606  끝
31663  끝
33892  끝
31682  끝
31745  끝
31734  끝
33893  끝
32293  끝
31721  끝
31723  끝
33900  끝
33890  끝
33894  끝
33895  끝
31628  끝
31666  끝
32452  끝
31743  끝
33896  끝
31634  끝
32656  끝
31547  끝
32516  끝
33906  끝
33631  끝
31816  끝
32000  끝
32763  끝
31670  끝
32221  끝
32011  끝
31667  끝
32030  끝
31744  끝
31614  끝
31605  끝
31623  끝
31668  끝
32004  끝
31665  끝
32190  끝
32143  끝
32126  끝
32496  끝
33897  끝
33611  끝
32155  끝
31707  끝
31603  끝
33915  끝
31692  끝
31622  끝
33901  끝
31726  끝
32754  끝
33898  끝
31620  끝
32164  끝
32121  끝
32655  끝
32127  끝
32167  끝
33707  끝
33902  끝
33862  끝
32128  끝
33695  끝
31747  끝
31742  끝
3